In [1]:
import os
import csv
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.corpus import opinion_lexicon
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import pandas as pd
import sqlite3

In [2]:
# Parsing은 text 받아오면서 끝냄. Sqlite DB에 있는 text 그대로 가져오면 사용 가능함

lexicon_dir = ""

# lexicon 가져오기 ( lexicon: {'word1' : 'score1', 'word2' : 'score2', ... } )
def load_lm_lexicon():
    # Loughran McDonald
    fn = os.path.join(lexicon_dir, "LoughranMcDonald_MD2018.csv")
    
    # ordered dictionary. 
    # Example: [('Word', 'AARDVARK'), ('Sequence Number', '1'), ('Word Count', '277'), ('Word Proportion', '1.48E-08'), 
    # ('Average Proportion', '1.24E-08'), ('Std Dev', '3.56E-06'), ('Doc Count', '84'), ('Negative', '0'), ('Positive', '0'), 
    # ('Uncertainty', '0'), ('Litigious', '0'), ('Constraining', '0'), ('Superfluous', '0'), ('Interesting', '0'), ('Modal', '0'), 
    # ('Irr_Verb', '0'), ('Harvard_IV', '0'), ('Syllables', '2'), ('Source', '12of12inf')]
    reader = csv.DictReader(open(fn))
    
    words2weights = {}
    
    for r in reader:
        # Assume: positive != 0 --> positive word, negative != 0 --> negative word
        pos_score = 1. if r['Positive'] != "0" else 0.
        neg_score = 1. if r['Negative'] != "0" else 0.
        sentiment_score = pos_score - neg_score
        
        # upper -> lower
        w = r['Word'].lower()
        
        # positive / negative labeling
        if sentiment_score:
            words2weights[w] = sentiment_score
    return words2weights

In [3]:
def load_hl_lexicon():
    # Hu and Liu 2004 opinion lexicon : nltk 내장
    words2weights = {w: 1.0 for w in opinion_lexicon.positive()}
    words2weights.update({w: -1.0 for w in opinion_lexicon.negative()})
    return words2weights

In [4]:
lexicon_dir = ""

def load_news_vader_lexicon():
    # 샌프란시스코 연준의 measuring news sentiment 논문에서 가공한 vader lexicon
    fn = os.path.join(lexicon_dir, "ns.vader.sentences.20k.csv")
    df = pd.read_csv(fn)
    words2weights = dict(zip(df['word'].values, df['sentiment'].values))
    return words2weights

In [5]:
def combine_lexicons(lexicons):
    # input(list): [lm_lexicon, hl_lexicon, vader lexicon]
    # and returns the union
    lexicons.reverse()
    words2weights = {}

    for lex in lexicons:
        for w in lex:
            words2weights.setdefault(w, 0.0)
            words2weights[w] += lex[w]
    
    return words2weights

In [6]:
def lexicon_scoring(text, lexicon):
    # text 받아온 것 전부 token화 (단어 단위로 자른다는 느낌) 하고 이모티콘이나 문장 부호 같은 것 아니면 전부 소문자화
    words = TweetTokenizer(preserve_case=False).tokenize(text)
    # words에서 단어별로 체크 후 합치기. w.lower()만 하면 key 없어버리면 None이니 key 없으면 0.0 처리
    score = sum([lexicon.get(w.lower(), 0.0) for w in words])

    score = score/len(words)
    return score

In [7]:
NEGATION_WORDS = set(nltk.sentiment.vader.NEGATE)

def negated_lexicon_scoring(text, lexicon):
    words = TweetTokenizer(preserve_case=False).tokenize(text)
    score = 0.0
    
    for i, w in enumerate(words):
        # 문맥에 부정어 있는지 봐야 하니까 체크 중인 word 앞 3단어 체크
        context = words[max(0, i-3):i]
        nega_adjust = 1.0
        # negation word(부정어) 있음 (ex: bad가 not bad가 되어서 부정적인 뜻이 아니게 됨)
        if set(context) & NEGATION_WORDS:
            nega_adjust = -1.0
        score += (nega_adjust * lexicon.get(w.lower(), 0.0))
    score = score / len(words)
    
    return score

In [8]:
# news_sentiment db에 있는 식별자들과 news_text db에 있는 것들 비교해서 news_sentiment db에 없는 것만 추가할 것
con = sqlite3.connect('news_sentiment.db')
news_sentiment_data = pd.read_sql('select source, topic, title, publish_date, link, keywords, text from news_sentiment', con)
con.close()

In [9]:
news_sentiment_data

,source,topic,title,publish_date,link,keywords,text
0,yahoo.com,news,Iran vows revenge against Israel for strike on...,2021-04-12T10:11:31Z,https://news.yahoo.com/iran-vows-revenge-again...,deal//agreement//iran//natanz//talks//incident...,"Iranian President Hassan Rouhani, second from ..."
1,yahoo.com,news,"Georgia sheriff: 3 officers wounded, 1 suspect...",2021-04-12T11:21:10Z,https://news.yahoo.com/sheriffs-official-georg...,officers//john//times//york//followers//piper/...,The New York Times\n\nAbraham Piper became a s...
2,yahoo.com,news,"AP PHOTOS: From Moscow to Pacific, Russia glor...",2021-04-12T07:35:18Z,https://news.yahoo.com/ap-photos-moscow-pacifi...,near//moscow//russia//glorifies//statue//photo...,MOSCOW (AP) — From a giant statue towering ove...
3,yahoo.com,news,Biden hasn't actually lifted Trump's historica...,2021-04-12T10:21:00Z,https://news.yahoo.com/biden-hasnt-actually-li...,note//law//biden//lifted//advocates//genocide/...,National Review\n\nSecretary of State Antony B...
4,yahoo.com,news,"Fiery crash kills driver, sends passenger to S...",2021-04-12T13:08:06Z,https://news.yahoo.com/fiery-crash-kills-drive...,reported//houston//patrol//men//passenger//cra...,Oxygen\n\nA serial killer hiding bodies in pla...
...,...,...,...,...,...,...,...
6799,macrobusiness.com.au,economics,"Politicians abandon Aussies, roll out welcome ...","Thu, 15 Apr 2021 14:15:11 +0000",https://www.macrobusiness.com.au/2021/04/polit...,pandemic//aussies//quarantine//governments//ro...,One of the most disturbing situations to arise...
6800,macrobusiness.com.au,economics,CoreLogic weekly house price update: still boo...,"Thu, 15 Apr 2021 14:10:50 +0000",https://www.macrobusiness.com.au/2021/04/corel...,major//increase//week//rising//weekly//values/...,CoreLogic’s daily dwelling values index regist...
6801,macrobusiness.com.au,economics,Links 16 April 2021,"Thu, 15 Apr 2021 14:01:20 +0000",https://www.macrobusiness.com.au/2021/04/links...,vary//16//trading//note//purchase//links//port...,Put the power of MacroBusiness into your portf...
6802,macrobusiness.com.au,economics,"Macro Afternoon: 15 April, 2021","Thu, 15 Apr 2021 07:41:54 +0000",https://www.macrobusiness.com.au/2021/04/macro...,vary//trading//macro//note//purchase//overlays...,Put the power of MacroBusiness into your portf...


In [10]:
con = sqlite3.connect('news_text.db')
news_text_data = pd.read_sql('select source, topic, title, publish_date, link, keywords, text from news_text', con)
con.close()

In [11]:
news_text_data

,source,topic,title,publish_date,link,keywords,text
0,yahoo.com,news,Iran vows revenge against Israel for strike on...,2021-04-12T10:11:31Z,https://news.yahoo.com/iran-vows-revenge-again...,deal//agreement//iran//natanz//talks//incident...,"Iranian President Hassan Rouhani, second from ..."
1,yahoo.com,news,"Georgia sheriff: 3 officers wounded, 1 suspect...",2021-04-12T11:21:10Z,https://news.yahoo.com/sheriffs-official-georg...,officers//john//times//york//followers//piper/...,The New York Times\n\nAbraham Piper became a s...
2,yahoo.com,news,"AP PHOTOS: From Moscow to Pacific, Russia glor...",2021-04-12T07:35:18Z,https://news.yahoo.com/ap-photos-moscow-pacifi...,near//moscow//russia//glorifies//statue//photo...,MOSCOW (AP) — From a giant statue towering ove...
3,yahoo.com,news,Biden hasn't actually lifted Trump's historica...,2021-04-12T10:21:00Z,https://news.yahoo.com/biden-hasnt-actually-li...,note//law//biden//lifted//advocates//genocide/...,National Review\n\nSecretary of State Antony B...
4,yahoo.com,news,"Fiery crash kills driver, sends passenger to S...",2021-04-12T13:08:06Z,https://news.yahoo.com/fiery-crash-kills-drive...,reported//houston//patrol//men//passenger//cra...,Oxygen\n\nA serial killer hiding bodies in pla...
...,...,...,...,...,...,...,...
8382,bostonglobe.com,news,Bridgette Mitchell named Northeastern women’s ...,"Sat, 17 Apr 2021 19:10:29 +0000",https://www.bostonglobe.com/2021/04/17/sports/...,bridgette//coach//university//mitchell//wagner...,Mitchell has coached women’s basketball since ...
8383,bostonglobe.com,news,State bans biomass power plants near environme...,"Sat, 17 Apr 2021 18:53:35 +0000",https://www.bostonglobe.com/2021/04/16/science...,state//communities//states//incentives//energy...,The move — a reversal from draft rules release...
8384,bostonglobe.com,news,Taking stock of the Bruins after the trade dea...,"Sat, 17 Apr 2021 18:26:23 +0000",https://www.bostonglobe.com/2021/04/17/sports/...,skill//capitals//bruins//draft//right//trade//...,The Bruins’ last pick in the top 20 was Urho V...
8385,bostonglobe.com,news,Harry and William were seen chatting together ...,"Sat, 17 Apr 2021 18:25:19 +0000",https://www.bostonglobe.com/2021/04/17/world/h...,brothers//family//chatting//walked//royal//har...,It was the first time the brothers had been to...


In [12]:
# sentiment score 저장하는 db인 news_sentiment에 포함되어 있지 않은 news_text_data만 점수 내서 집어넣을 것임
df_target = pd.merge(news_text_data, news_sentiment_data, how='outer', indicator='Exist')
df_target= df_target.loc[df_target['Exist']=='left_only']
df_target = df_target.loc[:, ['source','topic','title','publish_date','link','keywords','text']]
df_target = df_target.reset_index(drop=True)
#df_target

In [13]:
# combined lexicon
lex = combine_lexicons([load_lm_lexicon(), load_hl_lexicon(), load_news_vader_lexicon()])
# 부정어 조정 안 함
sent_score = []
# 부정어 조정함
sent_score_adj = []

for i in range(len(df_target)):
    try:
        # column index 6: news text
        sent_score.append(lexicon_scoring(df_target.iloc[i,6], lex))
        sent_score_adj.append(negated_lexicon_scoring(df_target.iloc[i,6], lex))
        #print(i)
    # text 없는 경우 sentiment score 0으로 대체
    except ZeroDivisionError as e:
        sent_score.append(0)
        sent_score_adj.append(0)
        #print(e)
            
news_text_data_with_sentiment_score = df_target
news_text_data_with_sentiment_score['sent_score'] = sent_score
news_text_data_with_sentiment_score['sent_score_adj'] = sent_score_adj

#news_text_data_with_sentiment_score

In [14]:
con = sqlite3.connect('news_sentiment.db')
news_text_data_with_sentiment_score.to_sql('news_sentiment', con, if_exists='append', index_label='id')
con.close()